<a href="https://colab.research.google.com/github/stoicelephant/DIDA340/blob/main/Copy_of_Final_Project_Dida_340.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# %env KAGGLE_USERNAME=jamesgiuf
# %env KAGGLE_KEY=c886c47a2c9510261eb39b5a38d569b9

# !kaggle datasets download -d mouadjaouhari/bitcoin-hourly-ohclv-dataset

# !unzip /content/bitcoin-hourly-ohclv-dataset.zip


In [18]:
import pandas as pd
import datetime as dt
bitcoin = pd.read_csv('https://raw.githubusercontent.com/stoicelephant/DIDA340/refs/heads/main/bitcoin_csv_simple')

In [19]:
# bitcoin.columns

In [20]:
bitcoin['dt'] = pd.to_datetime(bitcoin['DATETIME'])

In [21]:
# bitcoin[['DATETIME','OPEN','HIGH','CLOSE','LOW']].head(1)

In [22]:
# bitcoin.DATETIME[1][0:7]

In [23]:
# b1512 = bitcoin.loc[bitcoin.DATETIME.str.startswith('2015-12')]

In [24]:
# b1512.reset_index(drop=True)
# b1512

In [25]:
!pip install plotly

In [26]:
# import plotly.graph_objects as go


# fig = go.Figure(data=[go.Candlestick(x=b1512['DATETIME'],
#                 open=b1512['OPEN'],
#                 high=b1512['HIGH'],
#                 low=b1512['LOW'],
#                 close=b1512['CLOSE'])])

In [27]:
maxt= bitcoin.UNIX_TIMESTAMP.max()
mint= bitcoin.UNIX_TIMESTAMP.min()
week_secs = 60*60*24*7

range_dict = {}
count = 0

while mint <= maxt:
  inputs = [mint, mint+week_secs]
  mint = mint+week_secs
  range_dict[count] = inputs
  count +=1

Now i need to split the df into subframes

In [28]:
df_dict = {}
count = 0

for week in range_dict:
  count +=1
  current_range = range_dict[week]
  lhs = current_range[0]
  rhs = current_range[1]
  new_df =  bitcoin.loc[(bitcoin.UNIX_TIMESTAMP <= rhs) & (bitcoin.UNIX_TIMESTAMP > lhs)]
  df_dict[week] = new_df


In [29]:
num_weeks = len(range_dict) #565

In [30]:
!pip install --upgrade kaleido

In [31]:
!pip install matplotlib

In [32]:
# import plotly.graph_objects as go
# from PIL import Image
# import matplotlib.pyplot as plt
# import plotly.express as px

# num_weeks = len(range_dict)

# for i in range(num_weeks):
#   current_df = df_dict[i]
#   fig = go.Figure(data=[go.Candlestick(x=current_df['DATETIME'],
#                                        open=current_df['OPEN'],
#                                        high=current_df['HIGH'],
#                                        low=current_df['LOW'],
#                                        close=current_df['CLOSE'])])

#   fig.update_yaxes(dtick=20, showgrid=True, gridcolor="lightgray")
#   fig.write_image(f"fig_{i}.png")


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


Still trying to figure out a way on how to standardize thse images

In [33]:
num_weeks = len(range_dict) #565

In [34]:
df_dict = {}
count = 0

for week in range_dict:
  count +=1
  current_range = range_dict[week]
  lhs = current_range[0]
  rhs = current_range[1]
  new_df =  bitcoin.loc[(bitcoin.UNIX_TIMESTAMP <= rhs) & (bitcoin.UNIX_TIMESTAMP > lhs)]
  df_dict[week] = new_df

In [35]:
wpc_dict = {}
count = 0
for i in range(num_weeks-1):
  next_week_df = df_dict[i+1]
  open_next = next_week_df.OPEN.iloc[0]
  close_next = next_week_df.CLOSE.iloc[-1]
  diff_next = close_next - open_next
  wpc_dict[count] = {'open_next':open_next,'close_next':close_next,'diff_next':diff_next}
  count += 1

In [36]:
image_ref_df  = pd.DataFrame.from_dict(wpc_dict, orient='index')
image_ref_df = image_ref_df.reset_index()
image_ref_df  = image_ref_df.rename(columns = {'index':'week_number'})

In [37]:
def pos_neg(thing):
  if thing >= 0:
    return 1
  else:
    return 0

image_ref_df['indicator'] = image_ref_df.diff_next.apply(pos_neg)

In [38]:
image_ref_df

,week_number,open_next,close_next,diff_next,indicator
0,0,332.50,313.82,-18.68,0
1,1,313.82,270.16,-43.66,0
2,2,270.16,270.01,-0.15,0
3,3,270.01,209.49,-60.52,0
4,4,209.49,281.82,72.33,1
...,...,...,...,...,...
559,559,111124.25,116581.20,5456.95,1
560,560,116175.38,113551.56,-2623.82,0
561,561,112910.45,111837.72,-1072.73,0
562,562,111706.41,123655.69,11949.28,1


In [39]:
def image_label(image_path):
    image_path = image_path.strip('.png')
    image_path = image_path.strip('fig_')
    return image_path

In [41]:
from fastai import *
from fastai import visionlearner

In [42]:
stock_images = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items = get_image_files,
                 splitter = GrandparentSplitter(train_name='train', valid_name='valid'),
                 get_y = image_label,
                 item_tfms=Resize(224))

dls = stock_images.dataloaders(path, bs=256)

NameError: name 'DataBlock' is not defined

In [ ]:
learn = vision_learner(dls, models.resnet18, metrics=error_rate)
learn.fine_tune(2)